In [ ]:
from elasticsearch import helpers
from elasticsearch import Elasticsearch
from datetime import datetime
import json
from elasticsearch_reader import ElasticsearchReader as esr
from elasticsearch_creator import ElasticsearchCreator as esc

In [ ]:
with open("test_data.json","r") as file:
    _actions = json.loads(file.read())
result_list = list(esc.bulk_load(_actions))
bulk_result = esc.process_result(result_list)
bulk_result

In [ ]:
esr.get_last_id('inventory',"document")

In [ ]:
%%bash
curl -X GET "localhost:9200/_search?pretty" -H 'Content-Type: application/json' -d'
{
    "query": {
        "function_score": {
            "query": {
                "match": { "message": "elasticsearch" }
            },
            "script_score" : {
                "script" : {
                  "source": "Math.log(2 + doc[\u0027likes\u0027].value)"
                }
            }
        }
    }
}'

In [ ]:
%%bash
curl -X GET "localhost:9200/_search?pretty" -H 'Content-Type: application/json' -d'
{
    "query": {
        "match" : {
            "message" : "elasticsearch"
        }
    }
}'

In [ ]:
%%bash
curl -X GET "localhost:9200/_search?pretty" -H 'Content-Type: application/json' -d'
{
    "query": {
        "match_phrase" : {
            "message" : "elasticsearch"
        }
    }
}'

In [ ]:
def get_last_id(index, doctype):
    """Get the last ID for autoincremental ID generation"""
    _query = {
        "size": 1,
        "sort": {"created_at": "desc"},
        "query": {
            "match_all": {}
        }
    }
    _filter_path = 'hits.hits._id'
    try:
        result = es.search(
            index=index,
            doc_type=doctype,
            body=_query,
            filter_path=_filter_path)
        return int(result['hits']['hits'][0]['_id'])
    except ConnectionError as error:
        return 0
get_last_id('inventory', 'document')